Pre-processing the NICT JLE dataset

In [47]:
import re
from os import listdir


def generateCleanFile(filename):
    input = open("NICT_JLE_4.1/LearnerOriginal/" + filename, "r", encoding="latin1") 
    output = open("preProcessedData/" + filename, "w", encoding="latin1")
    lines = input.readlines()
    for line in lines:
        if '<B>' in line :
            output.write(removeInternalTags(line[3:len(line)-5])+"\n")
    input.close()
    output.close() 

def removeInternalTags(line):
    doubleTagPattern = "<.*?>(.+?)</.*?>"
    singleTagPattern = "</?.*?>"
    line = re.sub(doubleTagPattern, "", line)
    line = re.sub(singleTagPattern, "", line)
    return re.sub(r"\s+", " ", line)

files = [f for f in listdir("NICT_JLE_4.1/LearnerOriginal")]

for f in files:
    generateCleanFile(f)


Processing the dataset: extract features with the Bag of Word

Training the classifier to Predict SST

Accuracy and Confusion matrix